In [25]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [26]:
%hive_start

In [27]:
%timeout 300

In [28]:
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-10 04:30 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-10 04:37 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-10 05:07 /tmp/hivep4
drwxr-xr-x   - root supergroup          0 2020-02-10 04:38 /tmp/hivep5


In [29]:
!hdfs dfs -mkdir /tmp/hivep5

mkdir: `/tmp/hivep5': File exists


In [30]:
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-10 04:30 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-10 04:37 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-10 05:07 /tmp/hivep4
drwxr-xr-x   - root supergroup          0 2020-02-10 04:38 /tmp/hivep5


In [31]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep5/

copyFromLocal: `/tmp/hivep5/tbl0.csv': File exists


In [32]:
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep5/

copyFromLocal: `/tmp/hivep5/tbl1.csv': File exists


In [33]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 8.895 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.601 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.07 seconds


In [36]:
%%hive
DROP TABLE IF EXISTS pt5;
CREATE TABLE pt5
AS
SELECT YEAR(c4) as ano, c0
FROM tbl0 LATERAL VIEW explode(c5) adTable AS c0  

;
DROP TABLE IF EXISTS pt51
CREATE TABLE pt51
AS
SELECT ano, c0, count(1)
FROM pt5 
group by ano, c0

;


INSERT OVERWRITE DIRECTORY '/tmp/hivep5'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM pt51;

SELECT * FROM pt51;
Query ID = root_20200210051127_59db6624-f87a-40c0-81e8-8bc8005742ec
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1581309005799_0016, Tracking URL = http://b4ecbf811125:8088/proxy/application_1581309005799_0016/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581309005799_0016
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-10 05:11:37,229 Stage-1 map = 0%,  reduce = 0%
2020-02-10 05:11:42,704 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.53 sec
MapReduce Total cumulative CPU time: 1 seconds 530 msec
Ended Job = job_1581309005799_0016
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/hivep5/.hive-staging_hive_2020-02-10_05-11-27_676_9081366807505229275-1/-ext-10000
Moving data to directory /tmp/hi

In [24]:
!hdfs dfs -cat /tmp/hivep5/000000_0 

cat: `/tmp/hivep5/000000_0': No such file or directory


In [23]:
!hdfs dfs -copyToLocal /tmp/hivep5/000000_0 output;

copyToLocal: `output': File exists


In [38]:
%%hive
DROP TABLE may;
%hive_quit

%hive_quit
DROP TABLE may;
NoViableAltException(338@[])
	at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1300)
	at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:77)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:70)
	at org.apache.hadoop.hive.ql.Driver.compile(Driver.java:468)
	at org.apache.hadoop.hive.ql.Driver.compileInternal(Driver.java:1317)
	at org.apache.hadoop.hive.ql.Driver.runInternal(Driver.java:1457)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1237)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1227)
	at org.apache.hadoop.hive.cli.CliDriver.processLocalCmd(CliDriver.java:233)
	at org.apache.hadoop.hive.cli.CliDriver.processCmd(CliDriver.java:184)
	at org.apache.hadoop.hive.cli.CliDriver.processLine(CliDriver.java:403)
	at org.apache.hadoop.hive.cli.CliDriver.executeDriver(CliDriver.java:821)
	at org.apache.hadoop.hive.c